<a href="https://colab.research.google.com/github/young-hwanlee/gnn_examples/blob/main/2_gnn_graph_level_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!git clone https://github.com/young-hwanlee/gnn_examples.git

fatal: destination path 'gnn_examples' already exists and is not an empty directory.


In [59]:
# ## To avoid some errors (출처: https://life-is-potatoo.tistory.com/95 [삶은 감자:티스토리]) ==========
# ## 1st method
# import torch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def format_pytorch_version(version):
#     return version.split('+')[0]

# TORCH_version = torch.__version__
# TORCH = format_pytorch_version(TORCH_version)
# def format_cuda_version(version):
#     return 'cu' + version.replace('.', '')

# CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

## Ensure that the PyTorch and the PyG are the same version
# !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-geometric

## 2nd method
import os
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install torch_geometric

## Ensure that the PyTorch and the PyG are the same version
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install pytorch_metric_learning

## Both methods are too slow.

2.4.0+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [60]:
## Import required packages ====================================================================
# import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GATConv
from torch_geometric.nn.pool import global_mean_pool
from torch_geometric.loader import DataLoader

import numpy as np
import random
from sklearn.model_selection import train_test_split

In [61]:
## Set up the seed for reproducibility ============================================================
seed = 42

torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)     # if use multi-GPU

np.random.seed(seed)

random.seed(seed)

In [62]:
## Load a dataset ===============================================================================
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root="/tmp/PROTEINS", name="PROTEINS", transform=torch_geometric.transforms.ToDevice(device))

In [63]:
print(f'Dataset: {dataset}:')
print('============================================================================================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of labels: {len(dataset.y)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Classes: {torch.unique(dataset.y)}')
print(f'Number of y = 0: {len(np.array(torch.where(dataset.y == 0)).flatten())}')
print(f'Number of y = 1: {len(np.array(torch.where(dataset.y == 1)).flatten())}')
print('============================================================================================')
for i in range(0, len(dataset)):
    if i % 100 == 0:
        print(f'{i} : {dataset[i]}')

Dataset: PROTEINS(1113):
Number of graphs: 1113
Number of labels: 1113
Number of features: 3
Number of classes: 2
Classes: tensor([0, 1])
Number of y = 0: 663
Number of y = 1: 450
0 : Data(edge_index=[2, 162], x=[42, 3], y=[1])
100 : Data(edge_index=[2, 214], x=[50, 3], y=[1])
200 : Data(edge_index=[2, 216], x=[52, 3], y=[1])
300 : Data(edge_index=[2, 332], x=[96, 3], y=[1])
400 : Data(edge_index=[2, 114], x=[33, 3], y=[1])
500 : Data(edge_index=[2, 150], x=[43, 3], y=[1])
600 : Data(edge_index=[2, 180], x=[46, 3], y=[1])
700 : Data(edge_index=[2, 12], x=[4, 3], y=[1])
800 : Data(edge_index=[2, 188], x=[45, 3], y=[1])
900 : Data(edge_index=[2, 46], x=[11, 3], y=[1])
1000 : Data(edge_index=[2, 18], x=[5, 3], y=[1])
1100 : Data(edge_index=[2, 128], x=[36, 3], y=[1])


In [64]:
len(dataset)

1113

In [65]:
dataset[700].x

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [66]:
train_idx_tmp, test_idx = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=seed)

In [67]:
np.array(train_idx_tmp).T

array([ 381,  327,  997,  958,  582,  783,  737,  845,  296,  298,    2,
          6, 1036,  346, 1001,  247,  351,  767,  896,  215,  436,  539,
        485,  270,  448,  265, 1085,  930,  380,  332,  622,  643,  787,
        907,  618,  256,   25,  321,   47,  106,   55, 1077,  213,  120,
        451,  847,   72,  634,  359,  306,  409,  439,  596,  336,  941,
        786,  529,  704,  394,  584,  846,  430,   60,  551,   92, 1062,
        290, 1000,  280,  309,  945,  404,  182,  549,  319,  110,  411,
        668,  538,  493,  355,  365,  261,  352,  602,  712,  885,  137,
        601,  675,  909,  478,  673, 1081,   29,  165,  248,  307,  398,
        947,  342,  881,  851,  789,  593,  992,  588,   65,  458,  998,
        636,  254,  812,  312,  490,  221,  235,  688,  894,  305,  367,
        677, 1057,  249,  644,  590,   71,   94,  429,   33,  730,  259,
        204,  467, 1060,  445,   77,   84,  890,   82, 1084,  745,  732,
        731,  620,  435,  541,  350,  990,    5,  3

In [68]:
dataset_idx_tmp = train_idx_tmp[:2]
print(dataset_idx_tmp)

data_tmp = dataset[dataset_idx_tmp]

print(data_tmp[0])
print(data_tmp[1])

try:
    print(data_tmp[2])
except IndexError:
    pass

[381, 327]
Data(edge_index=[2, 276], x=[69, 3], y=[1])
Data(edge_index=[2, 250], x=[62, 3], y=[1])


In [69]:
data_tmp = dataset[:2]

print(data_tmp[0])
print(data_tmp[1])

try:
    print(data_tmp[2])
except IndexError:
    pass

Data(edge_index=[2, 162], x=[42, 3], y=[1])
Data(edge_index=[2, 92], x=[27, 3], y=[1])


In [111]:
train_dataset_tmp = dataset[train_idx_tmp]
train_idx, val_idx = train_test_split(list(range(len(train_dataset_tmp))), test_size=0.25, random_state=seed)

train_dataset = train_dataset_tmp[train_idx]
val_dataset = train_dataset_tmp[val_idx]
test_dataset = dataset[test_idx]

print(f'Number of train data: {len(train_dataset)}')
print(f'Number of val data: {len(val_dataset)}')
print(f'Number of test data: {len(test_dataset)}')

Number of train data: 667
Number of val data: 223
Number of test data: 223


In [112]:
# Create a mini-batch loaderl =======================================================================
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for graph in train_loader:
    print(graph)
    break
print(train_loader)
print(train_loader.dataset)

DataBatch(edge_index=[2, 12762], x=[3352, 3], y=[64], batch=[3352], ptr=[65])
PROTEINS(667)


In [115]:
# Create your GNN model ===========================================================================
class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(seed)
        # Choose between different GNN building blocks:
        self.conv1 = GATConv(dataset.num_features, 16)
        self.conv2 = GATConv(16, 16)
        self.lin = torch.nn.Linear(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = global_mean_pool(x, batch)

        return self.lin(x)

num_classes = dataset.y.max().item() + 1       # Assuming classes are indexed from 0
# model = GNN()
model = GNN().to(device)
print(model)

GNN(
  (conv1): GATConv(3, 16, heads=1)
  (conv2): GATConv(16, 16, heads=1)
  (lin): Linear(in_features=16, out_features=2, bias=True)
)


In [120]:
# Train and test your GNN model =======================================================================
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

def test(model, data_loader):
    loss, correct = 0, 0

    model.eval()
    for data in data_loader:
        data = data.to(device)
        out = model(data)
        loss += criterion(out, data.y).item() / len(data_loader)
        correct += torch.sum(out.argmax(dim=1) == data.y)

    acc = correct / len(data_loader.dataset)

    return loss, acc

def test(model, loader):
    model.eval()
    pred = model(test_dataset.x, test_dataset.edge_index).argmax(dim=1)
    correct = pred == test_dataset.y
    acc = int(correct.sum()) / int(test_dataset.test_mask.sum())

    return acc

def train(model, train_loader, n_epochs):
    model = model.to(device=device)
    best_val_loss = float('inf')
    best_model = None

    model.train()
    for epoch in range(n_epochs):
        train_loss, train_acc = 0, 0

        for data in train_loader:
            data = data.to(device)

            optimizer.zero_grad()       # Clear gradients
            out = model(data)
            loss = criterion(out, data.y)     # Compute the loss solely based on the training nodes
            train_loss += loss.item() / len(train_loader.dataset)
            train_acc += torch.sum(out.argmax(dim=1) == data.y) / len(train_loader.dataset)
            loss.backward()     # Derive gradients
            optimizer.step()     # Update parameters based on gradients

        if epoch % 10 == 0:
            val_loss, val_acc = test(model, val_loader)
            print(f'Epoch : {epoch:03d}, | Train loss : {train_loss:.4f}, | Train accuracy : {train_acc:.4f} | Valid loss : {val_loss:.4f}, | Valid accuracy : {val_acc:.4f}')
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model

    return best_model

In [121]:
best_model = train(model, train_loader, n_epochs=200)

print('\n')
test_loss, test_acc = test(best_model, data_loader=test_loader)
print(f'Accuracy: {test_acc:.4f}')

TypeError: GNN.forward() takes 2 positional arguments but 3 were given